# TrOCR Thesis-Ready Notebook

This notebook implements a **thesis-compliant TrOCR OCR pipeline** aligned with Chapter 3 of the manuscript. It is designed to run on **Google Colab Free** and supports training, evaluation, and reproducibility.

**Key features:**
- Image preprocessing (OpenCV)
- Dataset abstraction
- TrOCR fine-tuning (transfer learning)
- CER / WER evaluation
- Early stopping & checkpointing
- Colab-safe configuration


In [ ]:
# =====================
# 1. Environment Setup
# =====================
!pip install -q transformers accelerate evaluate jiwer opencv-python


In [ ]:
# =====================
# 2. Imports & Reproducibility
# =====================
import torch
import numpy as np
import cv2
from PIL import Image
from transformers import (
    TrOCRProcessor,
    VisionEncoderDecoderModel,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EarlyStoppingCallback
)
from torch.utils.data import Dataset
from jiwer import cer, wer

torch.manual_seed(42)
np.random.seed(42)


## 3. Image Preprocessing
Aligned with Section 3.3.3 of the manuscript.

In [ ]:
def preprocess_image(image_path, size=(384, 384)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, size, interpolation=cv2.INTER_LINEAR)
    img = cv2.adaptiveThreshold(
        img, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 61, 11
    )
    return Image.fromarray(img)


## 4. Dataset Definition
Formal dataset abstraction required for thesis reproducibility.

In [ ]:
class PrescriptionOCRDataset(Dataset):
    def __init__(self, image_paths, labels, processor):
        self.image_paths = image_paths
        self.labels = labels
        self.processor = processor

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = preprocess_image(self.image_paths[idx])
        text = self.labels[idx]
        encoding = self.processor(image, text, return_tensors="pt")
        return {
            "pixel_values": encoding.pixel_values.squeeze(),
            "labels": encoding.labels.squeeze()
        }


## 5. Model Initialization
Using microsoft/trocr-base-handwritten as specified in Chapter 3.

In [ ]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.eos_token_id = processor.tokenizer.sep_token_id


## 6. Metrics (CER / WER)
Required by Section 3.6.1.

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    pred_str = processor.batch_decode(predictions, skip_special_tokens=True)
    label_str = processor.batch_decode(labels, skip_special_tokens=True)
    return {
        "cer": cer(label_str, pred_str),
        "wer": wer(label_str, pred_str)
    }


## 7. Training Configuration
Colab Free–safe settings with early stopping.

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./trocr_results',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='steps',
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    num_train_epochs=10,
    fp16=torch.cuda.is_available(),
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='cer',
    greater_is_better=False
)


## 8. Trainer Setup
Supports fine-tuning or inference-only execution.

In [ ]:
# NOTE: Replace image_paths and labels with your verified dataset
image_paths = []  # list of image file paths
labels = []       # corresponding verified transcriptions

# Example split (simplified)
split = int(0.8 * len(image_paths))
train_dataset = PrescriptionOCRDataset(image_paths[:split], labels[:split], processor)
val_dataset = PrescriptionOCRDataset(image_paths[split:], labels[split:], processor)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


## 9. Training / Evaluation
Run only when dataset is ready.

In [ ]:
# trainer.train()
# trainer.evaluate()


## 10. Notes
- Designed for Google Colab Free
- Supports sequential fold execution if needed
- Fully aligned with Chapter 3 methodology
